# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.118099e+02     1.642166e+00
 * time: 0.11246705055236816
     1     1.074468e+01     8.590238e-01
 * time: 0.9155220985412598
     2    -1.190454e+01     9.865872e-01
 * time: 1.0468580722808838
     3    -3.455631e+01     7.578786e-01
 * time: 1.2141392230987549
     4    -4.802883e+01     5.519512e-01
 * time: 1.3661251068115234
     5    -5.707095e+01     1.985243e-01
 * time: 1.5077722072601318
     6    -5.981826e+01     1.165031e-01
 * time: 1.621873140335083
     7    -6.087506e+01     5.316547e-02
 * time: 1.7277112007141113
     8    -6.128578e+01     6.341996e-02
 * time: 1.8398540019989014
     9    -6.158668e+01     3.899764e-02
 * time: 1.9523930549621582
    10    -6.181262e+01     3.423000e-02
 * time: 2.0586540699005127
    11    -6.199566e+01     2.256229e-02
 * time: 2.1783711910247803
    12    -6.206332e+01     1.737270e-02
 * time: 2.2921810150146484
    13    -6.213241e+01     1.430989e-02
 * time: 2.3988530635

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671077
    AtomicLocal         -18.8557697
    AtomicNonlocal      14.8522648
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485384 
    Xc                  -19.3336825

    total               -62.261666458198
